In [30]:
import pandas as pd

gen_img_metadata = "image_generated/gen_img_metadata.csv"
captions_sample = "DeepFashion/captions_sample.csv"

prompt = "This gentleman is wearing a medium-sleeve shirt with pure color patterns. The shirt is with cotton fabric and its neckline is lapel. This gentleman wears a long trousers. The trousers are with cotton fabric and solid color patterns."
df_gen_img_metadata = pd.read_csv(gen_img_metadata)
image_key_mapped = df_gen_img_metadata[df_gen_img_metadata["prompt"] == prompt]["image_key"].unique()[0]
print("image_key: ", image_key_mapped)

# check if image_key_mapped exists in the captions_sample
df_captions_sample = pd.read_csv(captions_sample)
if image_key_mapped in df_captions_sample["image_key"].values:
    print(f"image '{image_key_mapped}' exists in captions_sample")


mapped_models = df_gen_img_metadata[df_gen_img_metadata["image_key"] == image_key_mapped]["model"].unique()
print("list of 'model' for the given 'image_key_mapped':")
print(mapped_models)


image_key:  MEN-Denim-id_00000089-28_1_front.jpg
image 'MEN-Denim-id_00000089-28_1_front.jpg' exists in captions_sample
list of 'model' for the given 'image_key_mapped':
['Ghibli' 'Ghibli_Metadata' 'StableDiffusion' 'StableDiffusion_Metadata'
 'CogView' 'CogView_Metadata' 'Flux' 'Flux_Metadata']
